In [38]:
import json
import torch
from transformers import (AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline)
import re
import numpy as np
import json
from sklearn.metrics.pairwise import cosine_similarity
import uuid
import chromadb
from chromadb.utils import embedding_functions
from collections import defaultdict
tokenizer = AutoTokenizer.from_pretrained("./models/hf-frompretrained-download/meta-llama/Meta-Llama-3-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("./models/hf-frompretrained-downloadmeta-llama/Meta-Llama-3-8B-Instruct")




Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.70s/it]


In [30]:
# Initialize the chromadb directory, and client.
db = chromadb.PersistentClient(path="./chroma_db_mpnet")
embedder = embedding_functions.SentenceTransformerEmbeddingFunction(
     model_name= 'all-mpnet-base-v2'
)
collection = db.get_or_create_collection(
    name=f"chroma_db_mpnet",
    embedding_function= embedder
)

⚠️ It looks like you upgraded from a version below 0.6 and could benefit from vacuuming your database. Run chromadb utils vacuum --help for more information.
c:\Users\Trung\anaconda3\envs\master_torch_gpu\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
#from transformers import pipeline
pipeline = pipeline("text-generation",
                    model = model,
                    tokenizer=tokenizer, 
                    eos_token_id=tokenizer.eos_token_id,
                    max_length=520,
                    num_return_sequences =1,
                    device_map = "auto",
                    )




In [5]:
def get_response(prompt):
    sequences = pipeline(
        prompt,
        max_length=2048,
        num_return_sequences=1,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id,
        truncation=True,
    )
    gen_text = sequences[0]["generated_text"]
    
    # Remove the input prompt from the generated text
    response = gen_text[len(prompt):].strip()
    
    return response

In [6]:
def create_user_profile(review):
    prompt = f"""
    You are an e-commerce recommender specialist. Your task is to create a comprehensive user profile based on the following reviews, listed in chronological order (oldest to newest):

    {review}

    Analyze this information and create a user profile following these steps:

1. Long-term preferences: Identify themes and valued attributes.
2. Short-term interests: Note recent preferences or emerging interests.
3. Demographic information: Infer age range, possible gender, and lifestyle.
4. User profile summary: Combine insights into a concise profile.

Present your analysis in a structured format, using clear headings for each section. Do not include any code in your response. Focus on creating a vivid picture of the user's preferences, habits, and potential future interests.The Created User Profile should not exceed 200 words """
    return get_response(prompt)



In [7]:
# Function to extract the latest 'n' reviews
def extract_latest_n_reviews(data, n):
    review = []

    for user in data:
        reviews = user['reviews']
        
        # Ensure reviews are sorted by timestamp (earliest to latest)
        sorted_reviews = sorted(reviews, key=lambda x: x['timestamp'])
        
        # Get the last 'n' reviews
        latest_reviews = sorted_reviews[-n:]
        
        # Append these reviews to the 'review' variable
        review.extend(latest_reviews)

    return review

In [8]:
# Read the input_set.json file
with open('input_set.json', 'r', encoding='utf-8') as f:
    input_set = json.load(f)

# Use the first user element from the input_set as an example
example_user = [input_set[0]]  # Wrap it in a list to pass as data

# Extract the latest 'n' reviews from the example user
n = 3
latest_reviews = extract_latest_n_reviews(example_user, n)

# Print the result
print("Latest Reviews from the example user:", latest_reviews)

Latest Reviews from the example user: [{'product_name': 'OGANA CELL Peptide Concentrating Amazing Lotion 2.03 fl.oz. (60ml) - 650ppm Peptide Contained Mild Moisturizing Facial Lotion, Hypoallergenic Natural Ingredients for Sensitive Skin', 'parent_asin': 'B07RBSLNFR', 'rating': 5.0, 'title': 'A little goes a long way!', 'text': 'This is a really nice moisturizing lotion. It goes on lightly and is readily absorbed into my skin. My skin feels amazingly softer and smoother and allows for a nice base for my makeup to be applied over. You only need a small pea sized amount to cover your entire face and neck. Highly recommend for all skin types, ages and for women as well as men.', 'timestamp': 1621184430697}, {'product_name': "Orange Peel Nature's Cleanse Facial Scrub - by Visage Pure - USDA Organic - Physician Formulated - Research Supported - Natural Exfoliating, Refreshing, Brightening, Skin Tightening Vitamin C Scrub. Cleans the Pores Deeply and Refreshes the Skin Bringing Back Its Glow

In [9]:
# Example usage

profile = create_user_profile(latest_reviews)
print(profile)

"""

**Long-term Preferences**

* Valued Attributes:
	+ Soft and healthy-looking skin
	+ Effective skincare routines
	+ Gentle, non-irritating ingredients
	+ Good value for money

**Short-term Interests**

* Recent interest in keratin-based hair care products
* Curiosity about post-Keratin treatment shampoos

**Writing Style and Sentiment**

* Positive and enthusiastic tone in reviews
* Use of descriptive language to convey satisfaction
* Emphasis on practical benefits and results
* No negative comments or complaints

**Demographic Information**

* Age Range: Late 20s to early 40s (based on product usage and concerns)
* Possible Gender: Female (due to focus on skincare and cosmetics)

**User Profile Summary**

Meet our user, Sarah! A young adult with a keen eye for effective skincare routines and gentle, natural ingredients. She values her time and budget, seeking good value for money. Her recent exploration of keratin-based hair care suggests she may prioritize hair health alongside h

In [6]:
profile = f"""**User Profile**
**Long-term Preferences**
The user values effectiveness, ease of use, and convenience when it comes to their skincare routine. They prioritize products that deliver noticeable results and appreciate simplicity.

**Short-term Interests**
Recent purchases suggest the user has become interested in anti-aging treatments and laser-based solutions. Their enthusiasm for NIRA's product bundle indicates they may explore other similar offerings.

**Writing Style and Sentiment**
The reviewer writes enthusiastically, with a strong emphasis on positive experiences. They employ descriptive language to convey satisfaction, indicating a high level of engagement. The text exudes confidence, suggesting the user trusts their own opinions.

**Demographic Information**
Based on writing style and content, we infer:
	* Age Range: Late twenties to early thirties
	* Possible Gender: Female
	* Lifestyle: Busy professional seeking efficient self-care routines

**Summary**: Our user profile reveals a proactive individual prioritizing effective skincare practices. With a focus on convenience and simplicity, she explores innovative solutions like laser technology. Her enthusiastic nature suggests loyalty towards brands offering seamless experience. As our data grows, expect her interests to shift toward more advanced, result-driven products."""

In [21]:
def create_preliminary_recommendations(user_profile):
    print("creating Prelim rec")
    prompt = f"""You are an expert recommendation system.

Based on the following user profile:

{user_profile}

Generate three general product categories that match the user's preferences and interests.

For each category, provide:

1. Name: **[Non-ambiguous product or category name]**
   Reason: [1-2 sentences explaining the fit]

**Guidelines:**

- Do not mention specific brands or product names, except if there is any preference mentioned in the User Profile.
- Use plain text with a numbered list.
- **Do not include any introductions, conclusions, or additional commentary.**
- **Do not repeat the guidelines or any part of this prompt in your response.**
- **Your entire response should consist only of the numbered list in the exact format specified.**

"""
    return get_response(prompt)


In [22]:
preliminary_rec = create_preliminary_recommendations(profile)
print(preliminary_rec)

creating Prelim rec
Here are three general product categories matching the user's preferences and interests:

1. **Natural Skincare Sets**: This category fits as it caters to Sarah's demand for soft and healthy-looking skin, gentle, non-irritating ingredients, and good value for money. The sets would likely contain cleansers, moisturizers, toners, and serums formulated from natural sources like plants, herbs, and essential oils.
2. **Keratin-Based Haircare Essentials**: Given Sarah's recent interest in keratin-based hair care products, this category aligns well with her needs. It could comprise shampoos, conditioners, masks, and styling tools designed specifically for nourishing and protecting damaged hair using keratin-rich formulas.
3. **Wellness-Inspired Bath & Body Products**: As Sarah appreciates gentle, natural ingredients and seeks relaxing experiences, these products might appeal to her. This category could encompass bath salts, body washes, lotions, and creams infused with cal

In [23]:
def extract_product_names(prompt):
    product_names = []

    # Use a regular expression to find all occurrences of text between ** and **
    product_names = re.findall(r'\*\*(.*?)\*\*', prompt)

    return product_names

In [27]:
# Extract product names from the prompt
product_names = extract_product_names(preliminary_rec)
# Output the product names
print(product_names)

['Natural Skincare Sets', 'Keratin-Based Haircare Essentials', 'Wellness-Inspired Bath & Body Products']


In [32]:
def query_and_print_results(product_names, collection):
    for product_name in product_names:
        # Execute the query for the current product name
        results = collection.query(
            query_texts=[product_name],  # Chroma will embed this for you
            n_results=10  # how many results to return
        )
        
        # Print results for the current product name
        print(f"Top 10 similar documents for '{product_name}':")
        for i, (doc_id, distance, document) in enumerate(zip(results['ids'][0], results['distances'][0], results['documents'][0]), 1):
            print()
            print(f"   {i}. Document ID: {doc_id}")
            print(f"      Distance: {distance:.4f}")
            print(f"      Document: {document}")
        print()




In [33]:
# Example usage with a product list
product_names = ['Natural Skincare Sets', 'Keratin-Based Haircare Essentials', 'Wellness-Inspired Bath & Body Products']

# Assuming you have your `collection` object already set up
# Call the function with the product names and collection object
query_and_print_results(product_names, collection)

Top 10 similar documents for 'Natural Skincare Sets':
   1. Document ID: 219b580f-6740-4acf-bf11-93c4e0b2173e
      Distance: 0.6144
      Document: Fifth & Skin Better'n Ur Skin Mineral Bronzer (TOUCH OF SUN) - Natural Face Bronzer - Gluten Free Tan - Cruelty Free Makeup - Vegan - Paraben Free
   2. Document ID: be692f89-3659-4813-94a8-59c7a2d8e8d3
      Distance: 0.6329
      Document: artnaturals Natural Face Makeup Primer - 1 Fl Oz Hydrating Facial Foundation - Long Lasting for Wrinkles, Oily Skin, and Pore Size - Rejuvenating Formula - Aloe Vera, Coconut Oil and Safflower Oil
   3. Document ID: 46b93a98-5259-4b11-ba3e-d2b9744c9bef
      Distance: 0.6334
      Document: Organic Self Tanner for Face and Body - Streak Free Sunless Tanner - Self Tanning Lotion with Manuka Honey and Aloe Vera - Fake Tan for Fair to Medium Skin - Indoor Tanning Lotion with Bronzer (8oz)
   4. Document ID: 9310d154-409c-4481-b989-f56bbe7d2597
      Distance: 0.6469
      Document: Ownest Matte Oil Contro

In [39]:
def collect_results_alternating_shortest(product_names, collection):
    doc_distance_map = defaultdict(list)  # To store documents (product names) and distances per product
    final_result = []
    
    # Step 1: Collect results for each product name
    for product_name in product_names:
        results = collection.query(
            query_texts=[product_name],  # Chroma will embed this for you
            n_results=10  # how many results to return
        )
        
        # Store product names (documents) and distances
        doc_distance_map[product_name] = list(zip(results['documents'][0], results['distances'][0]))

    # Step 2: Iteratively pick the shortest path from each list
    while len(final_result) < 10:
        for product_name in product_names:
            # Get the result list for the current product
            if product_name in doc_distance_map and doc_distance_map[product_name]:
                # Sort by shortest distance and pick the first one
                sorted_results = sorted(doc_distance_map[product_name], key=lambda x: x[1])
                
                # Take the closest result (first element) if not already in final_result
                closest_document, distance = sorted_results.pop(0)
                
                # Add the document to final_result if not already present
                if closest_document not in final_result:
                    final_result.append(closest_document)
                
                # Remove it from the map since we have processed it
                doc_distance_map[product_name] = sorted_results
            
            # Break the loop if we already have 10 elements in the final result
            if len(final_result) >= 10:
                break
    
    return final_result

In [41]:
# Example usage with a product list
product_names = ['Natural Skincare Sets', 'Keratin-Based Haircare Essentials', 'Wellness-Inspired Bath & Body Products']

# Assuming you have your `collection` object already set up
# Call the function with the product names and collection object
final_list = collect_results_alternating_shortest(product_names, collection)
# Output the final list
print("Final list of document IDs:", final_list)

Final list of document IDs: ["Fifth & Skin Better'n Ur Skin Mineral Bronzer (TOUCH OF SUN) - Natural Face Bronzer - Gluten Free Tan - Cruelty Free Makeup - Vegan - Paraben Free", 'Keratin Secrets Keratin Infused Shampoo and Conditioner Duo 12 Ounce', 'Bath and Body Works 2 Pack Aromatherapy Stress Relief Sage and Cedarwood Body Wash & Foam Bath 10 Oz.', 'artnaturals Natural Face Makeup Primer - 1 Fl Oz Hydrating Facial Foundation - Long Lasting for Wrinkles, Oily Skin, and Pore Size - Rejuvenating Formula - Aloe Vera, Coconut Oil and Safflower Oil', 'Keratin Cure Brazilian Bio Chocolate Professional Complex Coconut Argan Oil Aminos Blow Out Hair Treatment Formaldehyde Free (160ml/ 5 fl oz) tratamiento keratina', 'Bath and Body Works - Aromatherapy - Promotes Stress Relief- Cedarwood & Sage- Body Wash & Foam Bath & Body Cream – Bundle', 'Organic Self Tanner for Face and Body - Streak Free Sunless Tanner - Self Tanning Lotion with Manuka Honey and Aloe Vera - Fake Tan for Fair to Medium 